# Registros de empleados

Se creara una aplicacion grafica con Python que gestione una base de datos de "empleados", entendiendose por gestion la Conexión a la base de datos, Creación de la tabla de empleados,Inserción, Consulta, actualización y Eliminación de un empleado y su informacion dentro de la empresa.

In [19]:
### Importacion de librerias
import sqlite3
from sqlite3 import Error
from tkinter import *
from tkinter import messagebox

# Creación de la conexión a la base de datos

In [20]:
def crear_conexion():
    conn = None;
    try:
        conn = sqlite3.connect('empleados_db.sqlite')
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

# Creación de la tabla de empleados

In [21]:
def crear_tabla(conn):
    try:
        sql = ''' CREATE TABLE IF NOT EXISTS empleados (
                                        id integer PRIMARY KEY,
                                        nombre text NOT NULL,
                                        cargo text NOT NULL,
                                        salario real
                                    ); '''
        conn.cursor().execute(sql)
    except Error as e:
        print(e)

## Insercion de empleados

In [22]:
def insertar_empleado(conn, empleado):
    sql = ''' INSERT INTO empleados(nombre,cargo,salario)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, empleado)
    conn.commit()
    return cur.lastrowid

# Consulta de empleados

In [23]:
def consultar_empleados(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM empleados")
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Actualización de empleados

In [24]:
# Actualización de empleados
def actualizar_empleado(conn, empleado):
    sql = '''
    UPDATE empleados
    SET nombre = ?,
        cargo = ?,
        salario = ?
    WHERE id = ?
    '''
    cur = conn.cursor()
    try:
        cur.execute(sql, empleado)
        conn.commit()
        return "Empleado actualizado correctamente."
    except Exception as e:
        print(f"Error al actualizar empleado: {e}")
        return f"Error al actualizar empleado: {e}"


# Eliminación de un empleado

In [25]:
# Eliminación de un empleado
def eliminar_empleado(conn, id):
    sql = 'DELETE FROM empleados WHERE id = ?'
    cur = conn.cursor()
    try:
        cur.execute(sql, (id,))
        conn.commit()
        return "Empleado eliminado correctamente."
    except Exception as e:
        print(f"Error al eliminar empleado: {e}")
        return f"Error al eliminar empleado: {e}"

# Creación de la ventana de la aplicación

In [26]:
def crear_ventana():
    ventana = Tk()
    ventana.title("Gestión de empleados")
    ventana.geometry("800x600")
    return ventana

# Creación de botones y campos de texto para la interfaz de usuario

In [27]:
from tkinter import *
from tkinter import messagebox

import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QWidget, QVBoxLayout, QHBoxLayout, QLineEdit, QPushButton, QTextEdit
from PyQt5.QtGui import QTextCursor
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QMessageBox
from sqlite3 import Error
import sqlite3

def crear_conexion():
    conn = None
    try:
        conn = sqlite3.connect('empleados_db.sqlite')
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def crear_tabla(conn):
    try:
        sql = ''' CREATE TABLE IF NOT EXISTS empleados (
                                        id integer PRIMARY KEY,
                                        nombre text NOT NULL,
                                        cargo text NOT NULL,
                                        salario real
                                    ); '''
        conn.cursor().execute(sql)
    except Error as e:
        print(e)

def insertar_empleado(conn, empleado):
    sql = ''' INSERT INTO empleados(nombre,cargo,salario)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, empleado)
    conn.commit()
    return cur.lastrowid

def consultar_empleados(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM empleados")
    rows = cur.fetchall()
    for row in rows:
        print(row)

def actualizar_empleado(conn, empleado):
    sql = '''
    UPDATE empleados
    SET nombre = ?,
        cargo = ?,
        salario = ?
    WHERE id = ?
    '''
    cur = conn.cursor()
    cur.execute(sql, empleado)
    conn.commit()

def eliminar_empleado(conn, id):
    sql = 'DELETE FROM empleados WHERE id = ?'
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()

class VentanaPrincipal(QMainWindow):
    def __init__(self, conn):
        super().__init__()
        self.conn = conn
        self.setWindowTitle("Gestión de empleados")
        self.setGeometry(100, 100, 800, 600)

        widget_principal = QWidget()
        layout_principal = QVBoxLayout()
        widget_principal.setLayout(layout_principal)

        self.nombre_texto = QLineEdit()
        self.cargo_texto = QLineEdit()
        self.salario_texto = QLineEdit()
        self.resultados_texto = QTextEdit()
        self.id_texto = QLineEdit()

        layout_principal.addWidget(QLabel("Nombre del empleado:"))
        layout_principal.addWidget(self.nombre_texto)

        layout_principal.addWidget(QLabel("Cargo del empleado:"))
        layout_principal.addWidget(self.cargo_texto)

        layout_principal.addWidget(QLabel("Salario del empleado:"))
        layout_principal.addWidget(self.salario_texto)

        layout_principal.addWidget(QPushButton("Insertar empleado", clicked=self.insertar_empleado))

        layout_principal.addWidget(QLabel("Resultados de la consulta:"))
        layout_principal.addWidget(self.resultados_texto)

        layout_principal.addWidget(QPushButton("Consultar empleados", clicked=self.mostrar_empleados))

        layout_principal.addWidget(QLabel("ID del empleado a actualizar/eliminar:"))
        layout_principal.addWidget(self.id_texto)

        layout_principal.addWidget(QPushButton("Cargar empleado", clicked=self.cargar_empleado))
        layout_principal.addWidget(QPushButton("Actualizar empleado", clicked=self.actualizar_empleado))
        layout_principal.addWidget(QPushButton("Eliminar empleado", clicked=self.eliminar_empleado))

        self.setCentralWidget(widget_principal)

    def insertar_empleado(self):
        nombre = self.nombre_texto.text()
        cargo = self.cargo_texto.text()
        salario = float(self.salario_texto.text())
        empleado = (nombre, cargo, salario)
        empleado_id = insertar_empleado(self.conn, empleado)
        QMessageBox.information(self, "Información", f"Empleado insertado con ID: {empleado_id}")
        self.nombre_texto.clear()
        self.cargo_texto.clear()
        self.salario_texto.clear()

    def mostrar_empleados(self):
        cur = self.conn.cursor()
        cur.execute("SELECT * FROM empleados")
        rows = cur.fetchall()
        self.resultados_texto.clear()
        for row in rows:
            self.resultados_texto.insertPlainText(str(row) + '\n')

    def cargar_empleado(self):
        empleado_id = int(self.id_texto.text())
        cur = self.conn.cursor()
        cur.execute("SELECT * FROM empleados WHERE id=?", (empleado_id,))
        row = cur.fetchone()
        if row is not None:
            self.nombre_texto.setText(row[1])
            self.cargo_texto.setText(row[2])
            self.salario_texto.setText(str(row[3]))
        else:
            QMessageBox.warning(self, "Error", "No se encontró un empleado con ese ID")

    def actualizar_empleado(self):
        empleado_id = int(self.id_texto.text())
        respuesta = QMessageBox.question(self, "Confirmar actualización", "¿Estás seguro de que quieres actualizar este empleado?")
        if respuesta == QMessageBox.Yes:
            nombre = self.nombre_texto.text()
            cargo = self.cargo_texto.text()
            salario = float(self.salario_texto.text())
            empleado = (nombre, cargo, salario, empleado_id)
            actualizar_empleado(self.conn, empleado)
            QMessageBox.information(self, "Información", "Empleado actualizado.")
            self.nombre_texto.clear()
            self.cargo_texto.clear()
            self.salario_texto.clear()

    def eliminar_empleado(self):
        empleado_id = int(self.id_texto.text())
        respuesta = QMessageBox.question(self, "Confirmar eliminación", "¿Estás seguro de que quieres eliminar este empleado?")
        if respuesta == QMessageBox.Yes:
            eliminar_empleado(self.conn, empleado_id)
            QMessageBox.information(self, "Información", "Empleado eliminado.")
            self.nombre_texto.clear()
            self.cargo_texto.clear()
            self.salario_texto.clear()

if __name__ == '__main__':
    conn = crear_conexion()
    crear_tabla(conn)

    app = QApplication(sys.argv)
    ventana = VentanaPrincipal(conn)
    ventana.show()

    sys.exit(app.exec_())


ModuleNotFoundError: No module named 'PySide2'